In [9]:
library(dplyr)
library(readr)

setwd("C:/Users/s1155063404/Desktop/Projects/brazilian-ecommerce-dataset/DataPreprocessing")

#Load dataset
raw_geolocation = read_csv("../CleanedDataset/corrected_geolocation.csv")
raw_order = read_csv("../CleanedDataset/order.csv")
raw_product_measure = read_csv("../CleanedDataset/product_measure.csv")
raw_seller = read_csv("../CleanedDataset/seller.csv")

order = raw_order %>%
  select(order_id, product_id, order_products_value, order_freight_value, product_category_name, 
         customer_city, customer_state, customer_zip_code_prefix,
         order_purchase_timestamp, order_aproved_at, order_delivered_customer_date)


Parsed with column specification:
cols(
  zip_code_prefix = col_integer(),
  city = col_character(),
  state = col_character(),
  lat = col_double(),
  lng = col_double()
)
Parsed with column specification:
cols(
  .default = col_character(),
  order_products_value = col_double(),
  order_freight_value = col_double(),
  order_items_qty = col_integer(),
  order_sellers_qty = col_integer(),
  order_purchase_timestamp = col_datetime(format = ""),
  order_aproved_at = col_datetime(format = ""),
  order_estimated_delivery_date = col_datetime(format = ""),
  order_delivered_customer_date = col_datetime(format = ""),
  product_name_lenght = col_integer(),
  product_description_lenght = col_integer(),
  product_photos_qty = col_integer(),
  review_score = col_integer(),
  review_creation_date = col_datetime(format = ""),
  review_answer_timestamp = col_datetime(format = "")
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  product_id = col_character(),
 

## Feature Engineering

In [6]:
#Delivery time
order = order %>%
  mutate(delivery_time = as.numeric(order_delivered_customer_date - order_purchase_timestamp, units="days"),
         order_purchase_timestamp = NULL,
         order_aproved_at = NULL,
         order_delivered_customer_date = NULL) %>%
  filter(! is.na(delivery_time))

#product_size & weights
product = raw_product_measure %>%
  mutate(size = product_length_cm * product_height_cm * product_width_cm,
         weight = product_weight_g) %>%
  select(product_id, size, weight)

order = order %>% 
  left_join(product, by="product_id") %>%
  filter(! is.na(size)) %>%
  mutate(product_category_name = NULL)

In [7]:
#------------------------------------------------------------------------------------------------------
#Compute distance between seller and customer based on (zip_code, city)
#------------------------------------------------------------------------------------------------------
#Compute the geographical center of each (zip_code, city)
geolocation = raw_geolocation %>%
  group_by(zip_code_prefix, city) %>%
  summarize(lat = mean(lat),
            lng = mean(lng))

#Incorporate the location information
order = order %>%
  mutate(customer_city = tolower(customer_city),
         customer_zip_code_prefix = as.integer(customer_zip_code_prefix)) %>%
  left_join(geolocation %>%
              ungroup() %>%
              mutate(customer_city = tolower(city),
                     customer_zip_code_prefix = zip_code_prefix,
                     customer_lat = lat,
                     customer_lng = lng) %>%
              select(starts_with("customer")), by=c("customer_zip_code_prefix", "customer_city")) %>%
  filter(!is.na(customer_lat))

order = order %>%
  left_join(raw_seller, by=c("order_id", "product_id")) %>%
  mutate(seller_city = tolower(seller_city),
         seller_zip_code_prefix = as.integer(seller_zip_code_prefix)) %>%
  left_join(geolocation %>%
              ungroup() %>%
              mutate(seller_city = tolower(city),
                     seller_zip_code_prefix = zip_code_prefix,
                     seller_lat = lat,
                     seller_lng = lng) %>%
              select(starts_with("seller")), by=c("seller_zip_code_prefix", "seller_city")) %>%
  filter(!is.na(seller_lat))

In [8]:
#Convert location to distance on Earth; Using Spherical Law of Cosines to compute distance
distance <- function(lat1, lng1, lat2, lng2){
  R = 6371 #Radius of Earth in km
  dang = sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng2 - lng1)
  return(R * acos(pmax(pmin(dang, 1), -1)))
}
ang2rad <- function(ang){
  return(ang * pi / 180)
}
order = order %>% 
  mutate(distance = distance(ang2rad(customer_lat), ang2rad(customer_lng),
                             ang2rad(seller_lat), ang2rad(seller_lng)),
         customer_lat = NULL,
         customer_lng = NULL,
         seller_lat = NULL,
         seller_lng = NULL)

order_id,product_id,order_products_value,order_freight_value,customer_city,customer_state,customer_zip_code_prefix,delivery_time,size,weight,seller_id,seller_zip_code_prefix,seller_city,seller_state,distance
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<dbl>
b95df3cef5297e79ef709ba256518f6f,6cdd53843498f92890544667809f1595,349.90,13.84,guaratuba,PR,832,5.739851,11400,900,ccc4bbb5f32a6ab2b7066a4130f114e3,803,curitiba,PR,86.17268
59af46052a799e80e2f0c665c587731d,ae5cad88462eb7b7b61401e31c45618e,15.00,15.10,uberlandia,MG,384,4.017214,19200,250,ce27a3cc3c8cc1ea79d11e561e9bebb6,30,sao paulo,SP,542.41402
a3e6136894621db402a772c6bc72a12a,0c9ff9d8ed9b9bdd825487b3a66e05f5,238.90,18.00,itapevi,SP,66,6.946479,48373,7950,8581055ce74af1daba164fdbd55a40de,71,guarulhos,SP,48.62742
b675ea5a618922f6e679e30531b8957b,ad0a798e7941f3a5a2fb8139cb62ad78,29.99,18.23,goiania,GO,743,23.095923,2560,550,1835b56ce799e6a4dc4eddc053f04066,149,ibitinga,SP,562.43999
195a8be6794c487fe6cfbb97b7c61902,eaf2046d4c87809247a30050ea13df03,295.99,47.65,recife,PE,511,14.449048,13851,3800,8bd0f31cf0a614c658f6763bd02dea69,12,sao paulo,SP,2123.85026
80dbbd6f2282898dbf912a2ec668175f,167752e318335bf6bdd6906c4c470266,160.00,19.00,rio grande,RS,962,15.414710,2016,200,7e1fb0a3ebfb01ffb3a7dae98bf3238d,144,franca,SP,1366.05927
